Example usage of FlowClient to run daily location, modal location and flow queries.

In [1]:
import flowclient

## Create an access token
Generate an API access token using [FlowAuth](../../flowclient/#flowauth).

## Create an API connection
Paste the access token from FlowAuth here as `TOKEN`

In [7]:
conn = flowclient.connect("http://localhost:9090", TOKEN)

/Users/jono/code/FlowKit/flowclient/flowclient/client.py:200: UserWarning: Communications with this server are NOT SECURE.
  return Connection(url, token, api_version, ssl_certificate)


## Daily location

### Specify query parameters

In [8]:
query_spec = flowclient.daily_location(
    date="2016-01-01",
    aggregation_unit="admin3",
    daily_location_method="last",
    subscriber_subset=None,
)

query_spec

{'query_kind': 'daily_location',
 'params': {'date': '2016-01-01',
  'aggregation_unit': 'admin3',
  'daily_location_method': 'last',
  'subscriber_subset': 'all'}}

### Set query running

In [9]:
query_id = flowclient.run_query(conn, query_spec)
query_id

'ddc61a04f608dee16fff0655f91c2057'

### Check status of query

In [10]:
flowclient.get_status(conn, query_id)

'Finished'

### Get query result

In [11]:
df = flowclient.get_result_by_query_id(conn, query_id)
df.head()

,pcod,total
0,524 3 09 50,18
1,524 5 13 67,17
2,524 1 03 13,20
3,524 5 14 73,24
4,524 3 07 37,14


Alternative (run query, wait for it to finish, and get result, in one function call):

In [12]:
df = flowclient.get_result(conn, query_spec)
df.head()

,pcod,total
0,524 3 09 50,18
1,524 5 13 67,17
2,524 1 03 13,20
3,524 5 14 73,24
4,524 3 07 37,14


## Modal location
### Specify query parameters
Either explicitly from a set of daily locations:

In [13]:
dl1 = flowclient.daily_location(
    date="2016-01-01",
    aggregation_unit="admin3",
    daily_location_method="last",
    subscriber_subset=None,
)
dl2 = flowclient.daily_location(
    date="2016-01-02",
    aggregation_unit="admin3",
    daily_location_method="last",
    subscriber_subset=None,
)
ml = flowclient.modal_location(dl1, dl2, aggregation_unit="admin3")

ml

{'query_kind': 'modal_location',
 'params': {'locations': ({'query_kind': 'daily_location',
    'params': {'date': '2016-01-01',
     'aggregation_unit': 'admin3',
     'daily_location_method': 'last',
     'subscriber_subset': 'all'}},
   {'query_kind': 'daily_location',
    'params': {'date': '2016-01-02',
     'aggregation_unit': 'admin3',
     'daily_location_method': 'last',
     'subscriber_subset': 'all'}}),
  'aggregation_unit': 'admin3'}}

Or from a date range:

In [14]:
ml = flowclient.modal_location_from_dates(
    "2016-01-01",
    "2016-01-03",
    aggregation_unit="admin3",
    daily_location_method="last",
    subscriber_subset=None,
)

ml

{'query_kind': 'modal_location',
 'params': {'locations': ({'query_kind': 'daily_location',
    'params': {'date': '2016-01-01',
     'aggregation_unit': 'admin3',
     'daily_location_method': 'last',
     'subscriber_subset': 'all'}},
   {'query_kind': 'daily_location',
    'params': {'date': '2016-01-02',
     'aggregation_unit': 'admin3',
     'daily_location_method': 'last',
     'subscriber_subset': 'all'}},
   {'query_kind': 'daily_location',
    'params': {'date': '2016-01-03',
     'aggregation_unit': 'admin3',
     'daily_location_method': 'last',
     'subscriber_subset': 'all'}}),
  'aggregation_unit': 'admin3'}}

### Set query running

In [15]:
query_id = flowclient.run_query(conn, ml)
query_id

'345b63248a346315d9518650a1cb01ac'

### Check status of query

In [16]:
flowclient.get_status(conn, query_id)

'Finished'

### Get query result

In [17]:
df = flowclient.get_result_by_query_id(conn, query_id)
df.head()

,pcod,total
0,524 3 09 50,15
1,524 5 13 67,13
2,524 1 03 13,13
3,524 5 14 73,9
4,524 3 07 37,14


Alternative:

In [18]:
df = flowclient.get_result(conn, ml)
df.head()

,pcod,total
0,524 3 09 50,15
1,524 5 13 67,13
2,524 1 03 13,13
3,524 5 14 73,9
4,524 3 07 37,14


## Flow
### Specify query parameters
Specify parameters for two location queries:

In [19]:
aggregation_unit = "admin1"
dl_past = flowclient.daily_location(
    "2016-01-01",
    aggregation_unit=aggregation_unit,
    daily_location_method="last",
    subscriber_subset=None,
)
dl_present = flowclient.daily_location(
    "2016-01-07",
    aggregation_unit=aggregation_unit,
    daily_location_method="last",
    subscriber_subset=None,
)

Specify parameters for flow query:

In [20]:
flow = flowclient.flows(dl_past, dl_present, aggregation_unit=aggregation_unit)
flow

{'query_kind': 'flows',
 'params': {'from_location': {'query_kind': 'daily_location',
   'params': {'date': '2016-01-01',
    'aggregation_unit': 'admin1',
    'daily_location_method': 'last',
    'subscriber_subset': 'all'}},
  'to_location': {'query_kind': 'daily_location',
   'params': {'date': '2016-01-07',
    'aggregation_unit': 'admin1',
    'daily_location_method': 'last',
    'subscriber_subset': 'all'}},
  'aggregation_unit': 'admin1'}}

### Set query running

In [21]:
query_id = flowclient.run_query(conn, flow)
query_id

'5417b6376ac6d2e0b7f69371f9a720cb'

### Check status of query

In [22]:
flowclient.get_status(conn, query_id)

'Finished'

### Get query result

In [23]:
df = flowclient.get_result_by_query_id(conn, query_id)
df.head()

,count,pcod_from,pcod_to
0,3,524 1,524 5
1,24,524 1,524 4
2,22,524 1,524 3
3,2,524 1,524 2
4,8,524 1,524 1


## Counts of Events by Location

In [26]:
df = flowclient.get_result(conn, flowclient.location_event_counts("2016-01-01", "2016-01-08", "admin3", "hour"))
df.head()

,date,hour,pcod,total
0,2016-01-01,0.0,524 1 01 04,1.0
1,2016-01-01,1.0,524 1 01 04,5.0
2,2016-01-01,2.0,524 1 01 04,4.0
3,2016-01-01,3.0,524 1 01 04,2.0
4,2016-01-01,4.0,524 1 01 04,3.0
